# COG USAGE

## 1 - Configuration

### 1.1 - Imports

In [ ]:
import sys
sys.path.append("../lib") 
import gdal
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy as np
import math
import getpass
from mundilib import MundiCatalogue, get_node
from utils import city_polygon_bbox, country_polygon_bbox
import xml.etree.ElementTree as ET
import cv2 as cv
from datetime import datetime, timedelta
import folium

### 1.2 - Use your acces key

In [ ]:
print("Please enter your acces key: ")
ak = input()
print("Please enter your secret key: ")
sk = input()

gdal.SetConfigOption("AWS_ACCESS_KEY_ID", ak)
gdal.SetConfigOption("AWS_SECRET_ACCESS_KEY", sk)
gdal.SetConfigOption("AWS_REGION", "eu-de")
gdal.SetConfigOption("AWS_VIRTUAL_HOSTING", "FALSE")
gdal.SetConfigOption("AWS_S3_ENDPOINT", "obs.eu-de.otc.t-systems.com")
gdal.SetConfigOption("GDAL_HTTP_UNSAFESSL", "YES")
gdal.SetConfigOption("AWS_DEFAULT_REGION", "eu-de")

### 1.3 - Select Bounding Box

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

lonmin = widgets.FloatText(value = 1.40,  description = 'Long min:', disabled = False, step = 0.01)
latmin = widgets.FloatText(value = 43.55, description = 'Lat min:',  disabled = False, step = 0.01)
lonmax = widgets.FloatText(value = 1.50,  description = 'Long max:', disabled = False, step = 0.01)
latmax = widgets.FloatText(value = 43.65, description = 'Lat max:',  disabled = False, step = 0.01)

bbox_menu = widgets.VBox([lonmin, latmin, lonmax, latmax])
country_menu = widgets.Text(placeholder = 'Ex: Germany, Spain...')
cities_menu = widgets.Text(placeholder = 'Ex: Toulouse, Paris...')

children = [bbox_menu, 
            widgets.VBox([
                widgets.Label(value="Enter a country:"),
                country_menu
            ]),
            widgets.VBox([
                widgets.Label(value="Enter a city:"),
                cities_menu
            ])]

tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'Bounding Box')
tab.set_title(1, 'Country')
tab.set_title(2, 'Cities')

bbox = [1.40, 43.55, 1.50, 43.65]
out = widgets.Output()
button = widgets.Button(description = 'Validate')
vbox = widgets.VBox([button, out])

display(tab)
display(vbox)

def click(b):
    global bbox
    if (tab.selected_index == 0):
        bbox = [lonmax.value, latmax.value, lonmin.value, latmin.value]
    elif(tab.selected_index == 1):
        polygon, bbox = country_polygon_bbox(country_menu.value)
    elif(tab.selected_index == 2):
        polygon, bbox, place_name = city_polygon_bbox(cities_menu.value)
    lonmin.value = bbox[2]
    latmin.value = bbox[3]
    lonmax.value = bbox[0]
    latmax.value = bbox[1]
    with out:
        print(bbox)
button.on_click(click)

### 1.4 - Select in mundi catalog

In [ ]:
c = MundiCatalogue()
csw = c.get_collection("Sentinel1").mundi_csw()
t = datetime.utcnow() + timedelta(days = -6)
xml_string='''
<?xml version='1.0'?>
<GetRecords xmlns='http://www.opengis.net/cat/csw/2.0.2'
    xmlns:DIAS='http://tas/DIAS'
    xmlns:csw='http://www.opengis.net/cat/csw/2.0.2'
    xmlns:dc='http://purl.org/dc/elements/1.1/'
    xmlns:dct='http://purl.org/dc/terms/'
    xmlns:gml='http://www.opengis.net/gml'
    xmlns:ogc='http://www.opengis.net/ogc'
    xmlns:ows='http://www.opengis.net/ows'
    xmlns:xsi='http://www.w3.org/2001/XMLSchema-instance'
    service='CSW' version='2.0.2' maxRecords='50' startPosition='1' resultType='results'
    outputFormat='application/xml' outputSchema='csw:Record' xsi:schemaLocation='http://www.opengis.net/cat/csw/2.0.2/CSW-discovery.xsd'>
    <csw:Query typeNames='csw:Record'>
        <csw:ElementSetName>full</csw:ElementSetName>
        <csw:Constraint version='1.1.0'>
            <ogc:Filter xmlns:ogc="http://www.opengis.net/ogc">
                <csw:And>
                     <csw:Or>
                        <ogc:PropertyIsEqualTo>
                            <ogc:PropertyName>DIAS:onlineStatus</ogc:PropertyName>
                            <ogc:Literal>STAGING</ogc:Literal>
                        </ogc:PropertyIsEqualTo>
                        <ogc:PropertyIsEqualTo>
                            <ogc:PropertyName>DIAS:onlineStatus</ogc:PropertyName>
                            <ogc:Literal>ONLINE</ogc:Literal>
                        </ogc:PropertyIsEqualTo>
                    </csw:Or>
                    <ogc:PropertyIsEqualTo>
                        <ogc:PropertyName>DIAS:imageFileFormat</ogc:PropertyName>
                        <ogc:Literal>COG_GEOTIFF</ogc:Literal>
                    </ogc:PropertyIsEqualTo>
                    <ogc:PropertyIsGreaterThan>
                        <ogc:PropertyName>DIAS:sensingStartDate</ogc:PropertyName>
                        <ogc:Literal>{1}Z</ogc:Literal>
                    </ogc:PropertyIsGreaterThan>
                    <ogc:Intersects>
                        <ogc:PropertyName>DIAS:footprint</ogc:PropertyName>
                        <gml:Envelope xmlns:gml="http://www.opengis.net/gml" srsName="EPSG:4326">                        
                            <gml:lowerCorner>{0[2]} {0[3]}</gml:lowerCorner>
                            <gml:upperCorner>{0[0]} {0[1]}</gml:upperCorner>
                        </gml:Envelope>
                    </ogc:Intersects>
                 </csw:And>
            </ogc:Filter>
        </csw:Constraint>
        <ogc:SortBy xmlns:ogc="http://www.opengis.net/ogc">
            <ogc:SortProperty>
                <ogc:PropertyName>DIAS:sensingStartDate</ogc:PropertyName>
                <ogc:SortOrder>DESC</ogc:SortOrder>
            </ogc:SortProperty>
        </ogc:SortBy>
    </csw:Query>
</GetRecords>'''.format(bbox, t.isoformat(timespec='seconds'))
payload = xml_string.strip()
csw.get_records(xml = payload)
print("Number of dataset records: %d" % len(csw.records))
if(len(csw.records) == 0):
    raise(Exception("No image Found"))

product_menu = widgets.Dropdown(
    options=csw.records,
    description='Produit:',
    disabled=False,
    width='100%',
)
display(product_menu)

### 1.5 - Select a file

In [ ]:
import boto3

b = product_menu.value 
baseurl = get_node(b, "DIAS:archiveProductURI").text
baseurl.replace("https://mundiwebservices.com/dp", "https://obs.eu-de.otc.t-systems.com")

client = boto3.client(
    's3',
    aws_access_key_id=ak,
    aws_secret_access_key=sk,
    endpoint_url='https://obs.eu-de.otc.t-systems.com',
)
basekey = baseurl[54:]
key = basekey + "/manifest.safe"
bucket = baseurl[36:53]
res = client.get_object(Bucket=bucket, Key=key)
tree = ET.fromstring(res["Body"].read().decode('utf-8'))
root = tree
dataobjects = root.find("dataObjectSection")
files = {}
for i in dataobjects:
    if "repID" in i.attrib and i.attrib["repID"] == "s1Level1MeasurementSchema":
        files[i[0][0].attrib["href"].split("/")[-1]] = basekey + i[0][0].attrib["href"][1:]
files_menu = widgets.Dropdown(
    options=files,
    description='File:',
    disabled=False,
    width='100%',
)
scale_menu = widgets.IntSlider(
    value=2,
    min=0,
    max=10,
    step=1,
    description='Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(files_menu, scale_menu)

## 2 - Using GPC images

In [ ]:
a = gdal.Open('/vsis3/'+ bucket +"/"+files_menu.value)
scale = scale_menu.value
band1 = a.GetRasterBand(1)
if scale > 0 and band1.GetOverviewCount() > scale:
    band1 = a.GetRasterBand(1).GetOverview(scale)
print(band1.XSize, band1.YSize)

### 2.1 - Compute map part

In [ ]:
gcps = a.GetGCPs()
i = 0
correspondance = {}
xmax = ymax = 0
px = []
py = []
valx = []
valy = []
valz = []
for gcp in gcps:
    x = math.floor(gcp.GCPPixel)
    y = math.floor(gcp.GCPLine)
    px.append(gcp.GCPPixel)
    py.append(gcp.GCPLine)
    
    valx.append(gcp.GCPX)
    valy.append(gcp.GCPY)
    valz.append(gcp.GCPZ)
    correspondance[(x, y)] = (gcp.GCPX, gcp.GCPY, gcp.GCPZ)
    if(x > xmax):
        xmax = x
    if(y > ymax):
        ymax = y
minvalx = min(valx)
minvaly = min(valy)
maxvalx = max(valx)
maxvaly = max(valy)
boundingbox = [(minvalx, minvaly), (maxvalx, minvaly), (maxvalx, maxvaly), (minvalx, maxvaly)]
    
grid_z1 = interpolate.griddata((valx, valy), px+np.array(py)*1j , ([bbox[0], bbox[0], bbox[2], bbox[2]], [bbox[1], bbox[3], bbox[3], bbox[1]]), method='linear')
lats = [math.floor(i) if not math.isnan(i) else j for i, j in zip(np.real(grid_z1), [float('nan'), a.RasterXSize, float('nan'), 0])]
longs = [math.floor(i) if not i == 0 else j for i, j in zip(np.imag(grid_z1), [float('nan'), a.RasterYSize, float('nan'), 0])]

position = [np.nanmin(lats), np.nanmin(longs), np.nanmax(lats), np.nanmax(longs)]
size = [position[2] - position[0], position[3] - position[1]]
print(position, size)


### 2.2 -  Rotate the image

In [ ]:
grid = interpolate.griddata((px, py), valx + np.array(valy) * 1j, ([position[2], position[2], position[0], position[0]], [position[3], position[1], position[1], position[3]]), method='linear')
position_scaled = [math.floor(i / 2 ** (scale + 1)) for i in position] if scale != 0 else position
size_scaled = [position_scaled[2] - position_scaled[0], position_scaled[3] - position_scaled[1]]

k = np.frombuffer(band1.ReadRaster1(position_scaled[0], position_scaled[1], size_scaled[0], size_scaled[1]),
                  dtype='uint16', count = size_scaled[0] * size_scaled[1]).reshape([size_scaled[1], size_scaled[0]])

rows, cols = k.shape

form1 = []
for i, j in zip(grid, boundingbox):
    t1, t2 = (np.real(i), np.imag(i))
    if math.isnan(t1):
        t1 = j[0]
        t2 = j[1]
    form1.append((t1, t2))

form2 = [(bbox[0], bbox[1]), (bbox[0], bbox[3]), (bbox[2], bbox[3]), (bbox[2], bbox[1])]

scalex, scaley = abs(rows / (form2[2][0] - form2[0][0])), abs(cols / (form2[2][1] - form2[0][1]))
scales = [(abs(rows / (form2[3][0] - form2[0][0])), abs(cols / (form2[1][1] - form2[0][1]))), 
          (abs(rows / (form2[2][0] - form2[1][0])), abs(cols / (form2[0][1] - form2[1][1]))), 
          (abs(rows / (form2[1][0] - form2[2][0])), abs(cols / (form2[3][1] - form2[2][1]))), 
          (abs(rows / (form2[0][0] - form2[3][0])), abs(cols / (form2[2][1] - form2[3][1])))]

corners = [[cols, 0], [cols, rows], [0, rows], [0, 0]]
form3 = [((j[0] - i[0]) * l[0] + k[0], (i[1] - j[1]) * l[1] + k[1]) for i, j, k, l in zip(form1, form2, corners, scales)]

pts1 = np.float32(corners)
pts2 = np.float32(form3)

M = cv.getPerspectiveTransform(pts1, pts2)
dst = cv.warpPerspective(np.flip(k, 1), M, (cols, rows), flags=cv.INTER_NEAREST)

width = 20
height = 16
plt.figure(figsize=(width, height))
plt.subplot(1, 3, 2)
plt.imshow(np.flip(k, 1), cmap='hot')
plt.subplot(1, 3, 3)
plt.imshow(dst, cmap='hot')
plt.show()

In [ ]:
print(boundingbox)

## 2.3 - Show the image on map

In [ ]:
m = folium.Map(location=(bbox[1], bbox[0]))
lats = [i[0] for i in form1]
longs = [i[1] for i in form1]
temp = [(np.min(longs), np.min(lats)), (np.max(longs), np.max(lats))]
temp2 = [[bbox[1], bbox[0]], [bbox[3], bbox[2]]]
def colmap(x):
    if x != 0:
        temp = np.log(x)
        return (temp, temp, temp, 1) 
    else:
        return (0, 0, 0, 0)

folium.raster_layers.ImageOverlay(
    image=dst,
    bounds=temp2,
    origin='upper',
    opacity=1,
    colormap=colmap
).add_to(m)
footprint = [correspondance[(0,0)], correspondance[(xmax, 0)], correspondance[(xmax, ymax)], correspondance[(0, ymax)]]
folium.vector_layers.Polygon(
    locations=[(i[1], i[0]) for i in footprint],
    color="lightblue",
).add_to(m)

folium.vector_layers.Polygon(
    locations=[(i[1], i[0]) for i in form2],
    color="pink",
).add_to(m)
folium.LayerControl().add_to(m)

m.save('/home/jovyan/work/folium_map.html')

# 3 - Download and georeferenced an image

## 3.1 - Download the image

In [ ]:
import threading
from ipywidgets import FloatProgress, HTML, VBox
from IPython.display import display
class DownloadProgress(object):
    def __init__(self, bucket, key, filename, client):
        self._filename = filename
        self._bucket = bucket
        self._key = key
        self._client = client
        self._size = self._client.head_object(Bucket=bucket, Key=key)["ContentLength"]
        self._seen_so_far = 0
        self._lock = threading.Lock()
        self.progress = FloatProgress(min=0, max=100, value=0)
        self.progress.bar_style = 'info'
        self.label = HTML()
        self.box = VBox(children=[self.label, self.progress])
        display(self.box)

    def callback(self, bytes_amount):
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = round((self._seen_so_far / self._size) * 100,2)
            self.progress.value = percentage
            self.label.value = u'{name}: {index} / {size}'.format(
                name=self._filename,
                index=self._seen_so_far,
                size=self._size
            )
    def download(self):
        self._client.download_file(Bucket=self._bucket, Key=self._key, Filename=self._filename, Callback=self.callback)

In [ ]:
file_name = files_menu.value.split("/")[-1]
dl = DownloadProgress(bucket, files_menu.value, file_name, client)
dl.download()

## 3.2 Georeferenced the image

In [ ]:
extension = file_name.split(".")[-1]
base_name = file_name[:-len(extension)-1]
dst_filename = "".join([base_name, "_ref.", extension])

# Opens source dataset
src_ds = gdal.Open(file_name)
ds = gdal.Warp('warp_test.tif', src_ds, format="GTIFF", dstSRS='EPSG:4326')
ds.BuildOverviews("average", [2,4,8,16])
co = ["TILED=YES", "BLOCKXSIZE=256", "BLOCKYSIZE=256", "COMPRESS=LZW", "GEOTIFF_KEYS_FLAVOR=ESRI_PE", "COPY_SRC_OVERVIEWS=YES"]
b = gdal.Translate(dst_filename, ds, format='GTIFF', noData=0, creationOptions=co)

ds = None
b = None
src_ds = None
print("Done")

## 3.3 - Compute localisation

In [ ]:
my_image = gdal.Open(dst_filename)
gt = my_image.GetGeoTransform()
scale = 0
band1 = my_image.GetRasterBand(1)
if scale > 0 and band1.GetOverviewCount() > scale:
    band1 = my_image.GetRasterBand(1).GetOverview(scale)

gt = [i for i in gt]
gt[1], gt[5] = (gt[1],gt[5]) if scale == 0 else (gt[1] * 2 ** (scale + 1), (gt[5] * 2 ** (scale + 1)))
print(gt)

In [ ]:
mx1 = bbox[2] 
my1 = bbox[3]
mx2 = bbox[0]
my2 = bbox[1]
px1 = int((mx1 - gt[0]) / gt[1])
py1 = int((my1 - gt[3]) / gt[5])
px2 = int((mx2 - gt[0]) / gt[1])
py2 = int((my2 - gt[3]) / gt[5])
position = [min(px1, px2), min(py1, py2), max(px1, px2), max(py1, py2)]
size = [position[2] - position[0], position[3] - position[1]]

k = np.frombuffer(band1.ReadRaster1(position[0], position[1], size[0], size[1]),
                  dtype='uint16', count = size[0] * size[1]).reshape([size[1], size[0]])

left = gt[0] + position[0]*gt[1]
top = gt[3] + position[1]*gt[5]
subset_extent = (left, left + size[0]*gt[1], 
                   top + size[1]*gt[5], top)
print(subset_extent)

## 3.4 - Show on map

In [ ]:
m = folium.Map(location=(subset_extent[2], subset_extent[0]))
def colmap(x):
    if x != 0:
        temp = np.log(x)
        return (temp, temp, temp, 1) 
    else:
        return (0, 0, 0, 0)
folium.raster_layers.ImageOverlay(
    image=k,
    bounds=[(subset_extent[2], subset_extent[0]), (subset_extent[3], subset_extent[1])],
    origin='upper',
    opacity=1,
    colormap=colmap
).add_to(m)
folium.vector_layers.Polygon(
    locations=[(subset_extent[2], subset_extent[0]), (subset_extent[3], subset_extent[0]), (subset_extent[3], subset_extent[1]), (subset_extent[2], subset_extent[1])],
    color="lightblue",
).add_to(m)
folium.LayerControl().add_to(m)
m.save('/home/jovyan/work/folium_map.html')

In [ ]:
from IPython.display import FileLink, FileLinks
FileLink('/home/jovyan/work/folium_map.html')

# 4 - Use georeferenced images on the test bucket

## 4.1 - Select the image

In [ ]:
file = "/vsis3/atos-only/test-cog/s1b-iw-grd-vh-20190702t174652-20190702t174717-016958-01fe85-002_ref.tiff"
#file = "/vsis3/atos-only/test-cog/s1b-iw-grd-vh-20190702t174627-20190702t174652-016958-01fe85-002_ref.tiff"
my_image = gdal.Open(file)
gt = my_image.GetGeoTransform()
scale = 1
band1 = my_image.GetRasterBand(1)
if scale > 0 and band1.GetOverviewCount() > scale:
    band1 = my_image.GetRasterBand(1).GetOverview(scale)
print(band1.XSize, band1.YSize)
gt = [i for i in gt]
gt[1], gt[5] = (gt[1],gt[5]) if scale == 0 else (gt[1] * 2 ** (scale + 1), (gt[5] * 2 ** (scale + 1)))
print(gt)

## 4.2 - Compute localisation

In [ ]:
mx1 = bbox[2] 
my1 = bbox[3]
mx2 = bbox[0]
my2 = bbox[1]
px1 = int((mx1 - gt[0]) / gt[1])
py1 = int((my1 - gt[3]) / gt[5])
px2 = int((mx2 - gt[0]) / gt[1])
py2 = int((my2 - gt[3]) / gt[5])
position = [min(px1, px2), min(py1, py2), max(px1, px2), max(py1, py2)]
#position = [0,0, 1881, 994 ]
size = [position[2] - position[0], position[3] - position[1]]
print(position, size)

k = np.frombuffer(band1.ReadRaster1(position[0], position[1], size[0], size[1]),
                  dtype='uint16', count = size[0] * size[1]).reshape([size[1], size[0]])

left = gt[0] + position[0]*gt[1]
top = gt[3] + position[1]*gt[5]
subset_extent = (left, left + size[0]*gt[1], 
                   top + size[1]*gt[5], top)
print(subset_extent)

## 4.3 - Show on map

In [ ]:
m = folium.Map(location=(subset_extent[2], subset_extent[0]))
def colmap(x):
    if x != 0:
        temp = np.log(x)
        return (temp, temp, temp, 1) 
    else:
        return (0, 0, 0, 0)
folium.raster_layers.ImageOverlay(
    image=k,
    bounds=[(subset_extent[2], subset_extent[0]), (subset_extent[3], subset_extent[1])],
    origin='upper',
    opacity=1,
    colormap=colmap
).add_to(m)
folium.vector_layers.Polygon(
    locations=[(subset_extent[2], subset_extent[0]), (subset_extent[3], subset_extent[0]), (subset_extent[3], subset_extent[1]), (subset_extent[2], subset_extent[1])],
    color="lightblue",
).add_to(m)
folium.LayerControl().add_to(m)
m.save('/home/jovyan/work/folium_map.html')